<a href="https://colab.research.google.com/github/chrisli-lcy/char-rnn.pytorch/blob/master/PULSE_7bv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/OpenMEDLab/PULSE-7bv5

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/OpenMEDLab/PULSE-7bv5)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="OpenMEDLab/PULSE-7bv5")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/810 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.22G [00:00<?, ?B/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("OpenMEDLab/PULSE-7bv5")
model = AutoModelForCausalLM.from_pretrained("OpenMEDLab/PULSE-7bv5")

In [ ]:
# 构建对话 prompt（模拟 ChatGPT 样式）
messages = [
    {"role": "user", "content": "Who are you?"},
    {"role": "assistant", "content": "I'm a helpful AI trained to assist you."},
    {"role": "user", "content": "What can you do?"}
]

# 拼接成 prompt 文本
prompt = ""
for msg in messages:
    if msg["role"] == "user":
        prompt += f"User: {msg['content']}\n"
    else:
        prompt += f"Assistant: {msg['content']}\n"
prompt += "Assistant: "  # 让模型从这里开始生成回应

# 编码输入
inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(inputs.input_ids.device)

# 生成回复
output = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    top_p=0.95,
    temperature=0.8,
    pad_token_id=tokenizer.eos_token_id
)

# 解码输出
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# 提取模型生成的最后一段回答
print(generated_text[len(prompt):].strip())